# 🔧 에이전트에서 MCP 활용하기


## MCP 사용하기

모델 컨텍스트 프로토콜(MCP)은 애플리케이션이 대규모 언어 모델(LLM)에 도구와 컨텍스트 데이터를 제공하는 방식을 정의하는 개방형 표준입니다. 이를 통해 외부 도구를 모델 워크플로우에 일관되고 확장 가능한 방식으로 통합할 수 있습니다. Microsoft Agent Framework는 모델 컨텍스트 프로토콜(MCP) 서버와의 통합을 지원하여 에이전트가 외부 도구 및 서비스에 액세스할 수 있도록 합니다


단, 타사의 MCP 서비스에 연결할 경우에는 비용, 보안 등의 고려사항이 있을 수 있기에 신중하게 활용해야 합니다. 


### MCP 도구 유형

에이전트 프레임워크는 세 가지 유형의 MCP 연결을 지원합니다.

#### MCPStdioTool - 로컬 MCP 서버

표준 입출력을 사용하여 로컬 프로세스로 실행되는 MCP 서버에 연결하는 데 MCPStdioTool을 사용합니다.

하기 코드에서는 Agent Framework(Python)과 함께 사용할 수 있는 로컬 MCP 서버 중 하나인 uvx mcp-server-calculator를 사용하여 간단한 수학 계산을 수행하는 예시를 보여줍니다.

In [ ]:
""" stdio로 로컬 MCP 서버를 사용하는 예제 """

async with (
    # 로컬 uvx mcp-server-calculator라는 MCP 서버를 MCPStdioTool로 연결합니다. 
    # 이 서버는 간단한 수학 계산을 수행하는 MCP 도구입니다.
    MCPStdioTool(
        name="calculator",
        command="uvx",
        args=["mcp-server-calculator"]
    ) as mcp_server,
    
    # 에이전트를 생성하며, mcp_server를 도구로 추가합니다. 
    # 또한, 에이전트는 Azure OpenAI Chat 모델을 사용하도록 설정되어 있습니다.
    Agent(
        client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        name="MathAgent",
        instructions="당신은 유용한 수학 도우미입니다. 계산을 해결할 수 있습니다.",
        tools=mcp_server
    ) as agent,
):
    result = await agent.run("15 * 23 + 45은 무엇입니까?")
    print(result)

# 현재 폴더에는 완성된 local_mcp_example.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\local_mcp_example.py

그러면 다음과 같은 결과가 나옵니다.

```bash
    (.venv) PS C:\GitHub\MAF_Workshop\MAF> python 1.py
    15 × 23 + 45의 값은 390입니다.
```

#### MCPStreamableHTTPTool - HTTP/SSE MCP 서버

SSE(Server-Sent Events)를 사용하여 HTTP를 통해 MCP 서버에 연결하려면 MCPStreamableHTTPTool을 사용합니다.

In [ ]:
"""HTTP 기반의 MCP 서버를 사용하는 예제입니다."""

async with (
    AzureCliCredential() as credential,
    MCPStreamableHTTPTool(
        name="Microsoft Learn MCP",
        url="https://learn.microsoft.com/api/mcp",
    ) as mcp_server,
    Agent(
        client=AzureAIAgentClient(credential=credential),
        name="DocsAgent",
        instructions="당신은 Microsoft Learn에 관한 질문에 도움을 주는 에이전트입니다.",
    ) as agent,
):
    result = await agent.run("Azure 스토리지 계정을 az cli를 사용하여 생성하는 방법은 무엇입니까?")
    print(result)

# 현재 폴더에는 완성된 http_mcp_example.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\http_mcp_example.py

그러면 다음과 같은 결과가 나옵니다.

```bash
    Azure 스토리지 계정을 **Azure CLI**를 사용하여 생성하려면 아래와 같은 단계를 따르면 됩니다.
    ---
    ### 1. Azure CLI 로그인
    ```sh
    az login
    ```
    브라우저가 열리고, 계정으로 로그인이 진행됩니다.
    ...... (생략)
```

#### MCPWebsocketTool - WebSocket MCP 서버

WebSocket 연결을 통해 MCP 서버에 연결하려면 MCPWebsocketTool을 사용합니다. 

다만, WebSocket 기반의 MCP 서버는 검색하기가 다소 막연하기 때문에 직접 구현하는 것이 테스트하기에 더 용이할 수 있습니다. 마침, 우리는 이번 과정의 후반에서 기존 에이전트를 MCP 서버로 전환하는 실습이 있는데, 그 실습에서 에이전트를 WebSocket 기반의 MCP 서버로 전환할 예정입니다. 그렇기에, 하기의 WebSocket 기반 MCP 도구 사용 예시는 해당 실습과 함께 실행해 보도록 하겠습니다. 코드 예시만 먼저 살펴보도록 하죠. 

In [ ]:
"""WebSocket 기반의 MCP 서버를 사용하는 예제입니다."""

async def websocket_mcp_example():

    async with (
        MCPWebsocketTool(
            name="restaurant-server",
            url="ws://localhost:8765",  # 실제 WebSocket MCP 서버 URL로 변경하세요
            load_prompts=False,         # 서버가 prompts/list를 지원하지 않으므로 비활성화
        ) as mcp_server,
        Agent(
            client=AzureOpenAIChatClient(credential=AzureCliCredential()),
            name="RestaurantAgent",
            instructions="당신은 레스토랑에서 서빙을 제공하는 에이전트입니다.",
            tools=mcp_server
        ) as agent,
    ):
        result = await agent.run("오늘의 스페셜 메뉴는 무엇입니까?")
        print(result)

# 현재 폴더에는 완성된 websocket_mcp_example.py 파일이 제공됩니다.
# 실행을 위해서는 새로운 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
# 그러나, 지금 실행해봐야 WebSocket MCP 서버가 아직 구현되지 않았기 때문에 연결에 실패할 것입니다.
# 해당 서버를 먼저 구현한 후에 그때 실행해 보도록 하죠.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\websocket_mcp_example.py

## MCP 제공하기

모델 컨텍스트 프로토콜(MCP)을 사용해서 에이전트를 도구로 노출하여 MCP 도구를 지원하는 다른 시스템에서 활용하는 방법을 알아봅니다.

---

### 📋 기존 에이전트를 MCP 서버로 노출

`as_mcp_server()` 메서드를 사용하면 에이전트를 MCP 서버로 변환할 수 있습니다. 이를 통해 MCP 호환 클라이언트에서 해당 에이전트를 도구로 호출할 수 있습니다.

먼저 MCP 서버로 노출할 에이전트를 생성합니다. 필요한 경우 Function 도구를 추가할 수도 있습니다.


In [ ]:
from typing import Annotated
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# 오늘의 스페셜 메뉴를 반환하는 도구
def get_specials() -> Annotated[str, "메뉴에서 스페셜 메뉴를 반환합니다. 🍽️✨"]:
    return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

# 메뉴 아이템의 가격을 반환하는 도구. 9.99 달러로 고정
def get_item_price(
    menu_item: Annotated[str, "메뉴 아이템의 이름입니다. 🍴"],
) -> Annotated[str, "메뉴 아이템의 가격을 반환합니다. 💰"]:
    return "$9.99"

# RestaurantAgent 에이전트 생성 및 도구 제공
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="RestaurantAgent",
    instructions="메뉴에 대한 질문에 답변합니다. 🍽️💬",
    tools=[get_specials, get_item_price],
)

### 🔄 에이전트를 MCP 서버로 전환

에이전트의 이름과 설명은 MCP 서버 메타데이터로 사용됩니다.

In [ ]:
# Expose the agent as an MCP server
server = agent.as_mcp_server()

### ▶️ MCP 서버 실행

아래 코드를 실행하여 MCP 서버를 시작합니다. 이 서버는 websocket 서버로 동작합니다.   
실행 후 MCP 호환 클라이언트(앞서 만든 websocket_mcp_example.py)에서 이 에이전트를 도구로 호출할 수 있습니다.

In [ ]:
async def mcp_asgi(scope, receive, send):
    if scope["type"] == "websocket":
        async with websocket_server(scope, receive, send) as (read_stream, write_stream):
            await server.run(read_stream, write_stream, server.create_initialization_options())

if __name__ == "__main__":
    print("🍴 Restaurant MCP 서버 시작: ws://localhost:8765")
    uvicorn.run(mcp_asgi, host="127.0.0.1", port=8765)

# 현재 폴더에는 완성된 restaurant_mcp_server.py 파일이 제공됩니다. 
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\restaurant_mcp_server.py

### 🧪 MCP 서버 테스트 

#### 📌 서버 실행

앞서 별도의 restaurant_mcp_server.py 서버를 실행하지 않았다면, 새로운 터미널에서 다음 명령어로 MCP 서버를 실행하세요:

```bash
    .\.venv\Scripts\Activate.ps1; 
    python .\MAF\restaurant_mcp_server.py
```

그러면, 터미널에 다음과 같이 서버가 시작되었다는 메시지가 출력됩니다:

![image.png](images/mcp-001.png)


#### 🔗 클라이언트 연결 

서버가 실행되었으니, 이제 MCP 클라이언트(예, 앞서 개발한 websocket_mcp_example.py)를 실행하여 MCP 서버에 연결하고, 에이전트가 도구로서 정상적으로 동작하는지 테스트합니다.

새로운 터미널에서 다음 명령어로 MCP 서버를 실행하세요:

```bash
    .\.venv\Scripts\Activate.ps1; 
    python .\MAF\websocket_mcp_example.py
```

그러면, 터미널에 다음과 같이 MCP를 호출한 결과가 출력됩니다:

![image.png](images/mcp-002.png)

